# Function Definitions and Imports

In [1]:
import math
import json
import requests
import itertools
import numpy as np
import time
import pandas as pd

from datetime import datetime, timedelta, timezone

In [2]:
# Get all the submissions for a specific subreddit in the given timeframe
# subreddit = string of subreddit to scrape
# after = epoch time (earliest submissions)
# before = epoch time (latest submissions)
def get_submissions(subreddit, after, before):
    
    url = ('https://api.pushshift.io/reddit/submission/search/?subreddit=' +  # Basic URL header
           str(subreddit) +  # Subreddit to scrape
           '&after=' + 
           str(after) +  # Scrape posts after (UTC format)
           '&before=' + 
           str(before) +  # Scrape posts after (UTC format)
           '&limit=1000')  # Can get at max 1000 submissions from pushshift at a time

    print(url)
    response = requests.get(url)
    submissions = json.loads(response.text)

    submissions = submissions['data']  # Only key in submissions is 'data'
    
    # Columns to include in submissions dataframe
    submission_columns = ['subreddit',     # Subreddit name
                          'id',            # Post ID
                          'created_utc',   # UTC time post was created
                          'title',         # Post title
                          'selftext',      # Post body
                          'num_comments',  # Number of comments on post
                          'score',         # Number of upvotes
                          'gilded'         # Number of silver/gold/platinum badges
                         ]
    
    # Create dataframe, where each row contains a submission
    submissions_df = pd.DataFrame(submissions).loc[:, submission_columns]

    while len(submissions) == 1000:
        
        after = submissions_df.created_utc.iloc[-1]  # Query from latest time of previous query
        print(after)
        
        url = ('https://api.pushshift.io/reddit/submission/search/?subreddit=' +  # Basic URL header
       str(subreddit) +  # Subreddit to scrape
       '&after=' + 
       str(after) +  # Scrape posts after (UTC format)
       '&before=' + 
       str(before) +  # Scrape posts after (UTC format)
       '&limit=1000')  # Can get at max 1000 submissions from pushshift at a time
        
        print(url)
        
        # Sometimes there is no data, or there's a moderator comment. In that case, pass
        try:
            response = requests.get(url)
            submissions = json.loads(response.text)
        

            submissions = submissions['data']  # Only key in submissions is 'data'

            print(len(submissions))  # Loop won't occur again if < 1000 submissions were found
                                     # since that means all submissions were queried

            # Append data to dataframe
            submissions_df = pd.concat([submissions_df, 
                                        pd.DataFrame(submissions).loc[:, submission_columns]],
                                      ignore_index=True)
        except:
            pass
        
    
    submissions_df.insert(2, 'created_time', np.nan)  # Create new column for time, with values initialized to NaN
    
    # Convert UTC time to datetime
    submissions_df['created_time'] = [datetime.utcfromtimestamp(utc).strftime('%Y-%m-%d %H:%M:%S') 
                                      for utc in submissions_df['created_utc']]
    
    return submissions_df

In [3]:
# Get list of comments for a submission
def get_submission_comments(submission_id):
    
    url = ('https://api.pushshift.io/reddit/comment/search/?link_id=' + 
           str(submission_id))
    
    print(url)
    response = requests.get(url)
    
    # Sometimes there is no data, or there's a moderator comment. In that case, pass
    try:
        comments_all_data = json.loads(response.text)['data']  # List of dicts

        # Get only the comment ('body') for each comment block
        # Comment block includes extraneous info (author, score, etc.)
        comments = [comment_block['body'] for comment_block in comments_all_data]

        return comments # Return list of comments
    except:
        pass

In [4]:
# Given list of comments, combine them into a single string
def combine_comments(comments):
    try:
        return ' '.join([str(elem) for elem in comments]) 
    except:
        pass

# Data Manipulation

__The following line of code is the only thing that has to be changed in this file:__

In [5]:
# Create DF with submissions between the following epoch times 
# Comments have not been added yet
submissions = get_submissions(subreddit='gadgets', 
                              after='2018-08-01', 
                              before='2019-01-01')


https://api.pushshift.io/reddit/submission/search/?subreddit=gadgets&after=2018-08-01&before=2019-01-01&limit=1000
1534682709
https://api.pushshift.io/reddit/submission/search/?subreddit=gadgets&after=1534682709&before=2019-01-01&limit=1000
1000
1536211998
https://api.pushshift.io/reddit/submission/search/?subreddit=gadgets&after=1536211998&before=2019-01-01&limit=1000
1000
1537507208
https://api.pushshift.io/reddit/submission/search/?subreddit=gadgets&after=1537507208&before=2019-01-01&limit=1000
1000
1539067881
https://api.pushshift.io/reddit/submission/search/?subreddit=gadgets&after=1539067881&before=2019-01-01&limit=1000
1000
1540505662
https://api.pushshift.io/reddit/submission/search/?subreddit=gadgets&after=1540505662&before=2019-01-01&limit=1000


/Users/samad/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


1000
1542112894
https://api.pushshift.io/reddit/submission/search/?subreddit=gadgets&after=1542112894&before=2019-01-01&limit=1000
1000
1543864699
https://api.pushshift.io/reddit/submission/search/?subreddit=gadgets&after=1543864699&before=2019-01-01&limit=1000
1000
1546058447
https://api.pushshift.io/reddit/submission/search/?subreddit=gadgets&after=1546058447&before=2019-01-01&limit=1000
79


In [6]:
submissions.head(1)

,subreddit,id,created_time,created_utc,title,selftext,num_comments,score,gilded
0,gadgets,93jv9u,2018-08-01 00:01:18,1533081678,New INSANE Inventions That Exist in 2018 - 201...,,0,1,0.0


In [7]:
# Create column 'comments' which contains list of comments for the submission
submissions['comments'] = submissions.id.apply(get_submission_comments)

https://api.pushshift.io/reddit/comment/search/?link_id=93jv9u
https://api.pushshift.io/reddit/comment/search/?link_id=93jvea
https://api.pushshift.io/reddit/comment/search/?link_id=93kffe
https://api.pushshift.io/reddit/comment/search/?link_id=93kg6i
https://api.pushshift.io/reddit/comment/search/?link_id=93kn3c
https://api.pushshift.io/reddit/comment/search/?link_id=93l0lg
https://api.pushshift.io/reddit/comment/search/?link_id=93lcaj
https://api.pushshift.io/reddit/comment/search/?link_id=93lnex
https://api.pushshift.io/reddit/comment/search/?link_id=93lre5
https://api.pushshift.io/reddit/comment/search/?link_id=93m916
https://api.pushshift.io/reddit/comment/search/?link_id=93mnec
https://api.pushshift.io/reddit/comment/search/?link_id=93mwbl
https://api.pushshift.io/reddit/comment/search/?link_id=93n0km
https://api.pushshift.io/reddit/comment/search/?link_id=93n5l9
https://api.pushshift.io/reddit/comment/search/?link_id=93nhvg
https://api.pushshift.io/reddit/comment/search/?link_id

https://api.pushshift.io/reddit/comment/search/?link_id=948k9v
https://api.pushshift.io/reddit/comment/search/?link_id=948lp3
https://api.pushshift.io/reddit/comment/search/?link_id=948nh3
https://api.pushshift.io/reddit/comment/search/?link_id=948q1r
https://api.pushshift.io/reddit/comment/search/?link_id=948ucz
https://api.pushshift.io/reddit/comment/search/?link_id=948v8q
https://api.pushshift.io/reddit/comment/search/?link_id=948viu
https://api.pushshift.io/reddit/comment/search/?link_id=948wk6
https://api.pushshift.io/reddit/comment/search/?link_id=948yb3
https://api.pushshift.io/reddit/comment/search/?link_id=9490ff
https://api.pushshift.io/reddit/comment/search/?link_id=94960p
https://api.pushshift.io/reddit/comment/search/?link_id=949cr0
https://api.pushshift.io/reddit/comment/search/?link_id=949cxr
https://api.pushshift.io/reddit/comment/search/?link_id=949epg
https://api.pushshift.io/reddit/comment/search/?link_id=949hi2
https://api.pushshift.io/reddit/comment/search/?link_id

https://api.pushshift.io/reddit/comment/search/?link_id=950g4k
https://api.pushshift.io/reddit/comment/search/?link_id=950nan
https://api.pushshift.io/reddit/comment/search/?link_id=9514ex
https://api.pushshift.io/reddit/comment/search/?link_id=95157w
https://api.pushshift.io/reddit/comment/search/?link_id=95180v
https://api.pushshift.io/reddit/comment/search/?link_id=951826
https://api.pushshift.io/reddit/comment/search/?link_id=951cgx
https://api.pushshift.io/reddit/comment/search/?link_id=951hqj
https://api.pushshift.io/reddit/comment/search/?link_id=951ipx
https://api.pushshift.io/reddit/comment/search/?link_id=951jii
https://api.pushshift.io/reddit/comment/search/?link_id=951s1q
https://api.pushshift.io/reddit/comment/search/?link_id=951yby
https://api.pushshift.io/reddit/comment/search/?link_id=9520og
https://api.pushshift.io/reddit/comment/search/?link_id=9527n1
https://api.pushshift.io/reddit/comment/search/?link_id=952an3
https://api.pushshift.io/reddit/comment/search/?link_id

https://api.pushshift.io/reddit/comment/search/?link_id=95kwzi
https://api.pushshift.io/reddit/comment/search/?link_id=95kxcs
https://api.pushshift.io/reddit/comment/search/?link_id=95kyvw
https://api.pushshift.io/reddit/comment/search/?link_id=95l14s
https://api.pushshift.io/reddit/comment/search/?link_id=95l4iq
https://api.pushshift.io/reddit/comment/search/?link_id=95l9r3
https://api.pushshift.io/reddit/comment/search/?link_id=95labo
https://api.pushshift.io/reddit/comment/search/?link_id=95llou
https://api.pushshift.io/reddit/comment/search/?link_id=95lr7v
https://api.pushshift.io/reddit/comment/search/?link_id=95lsf4
https://api.pushshift.io/reddit/comment/search/?link_id=95m2es
https://api.pushshift.io/reddit/comment/search/?link_id=95m3d9
https://api.pushshift.io/reddit/comment/search/?link_id=95m43u
https://api.pushshift.io/reddit/comment/search/?link_id=95m8jc
https://api.pushshift.io/reddit/comment/search/?link_id=95mi91
https://api.pushshift.io/reddit/comment/search/?link_id

https://api.pushshift.io/reddit/comment/search/?link_id=9620h1
https://api.pushshift.io/reddit/comment/search/?link_id=96217w
https://api.pushshift.io/reddit/comment/search/?link_id=962i2x
https://api.pushshift.io/reddit/comment/search/?link_id=962kcb
https://api.pushshift.io/reddit/comment/search/?link_id=962qn9
https://api.pushshift.io/reddit/comment/search/?link_id=9634hi
https://api.pushshift.io/reddit/comment/search/?link_id=963ay7
https://api.pushshift.io/reddit/comment/search/?link_id=963mli
https://api.pushshift.io/reddit/comment/search/?link_id=963rv7
https://api.pushshift.io/reddit/comment/search/?link_id=963v7x
https://api.pushshift.io/reddit/comment/search/?link_id=964jk1
https://api.pushshift.io/reddit/comment/search/?link_id=964jox
https://api.pushshift.io/reddit/comment/search/?link_id=964njk
https://api.pushshift.io/reddit/comment/search/?link_id=964o9p
https://api.pushshift.io/reddit/comment/search/?link_id=964tr8
https://api.pushshift.io/reddit/comment/search/?link_id

...

https://api.pushshift.io/reddit/comment/search/?link_id=a65k6o
https://api.pushshift.io/reddit/comment/search/?link_id=a65lmo
https://api.pushshift.io/reddit/comment/search/?link_id=a65rdv
https://api.pushshift.io/reddit/comment/search/?link_id=a65vyu
https://api.pushshift.io/reddit/comment/search/?link_id=a65xar
https://api.pushshift.io/reddit/comment/search/?link_id=a65y13
https://api.pushshift.io/reddit/comment/search/?link_id=a661br
https://api.pushshift.io/reddit/comment/search/?link_id=a663gg
https://api.pushshift.io/reddit/comment/search/?link_id=a66985
https://api.pushshift.io/reddit/comment/search/?link_id=a66cox
https://api.pushshift.io/reddit/comment/search/?link_id=a66i91
https://api.pushshift.io/reddit/comment/search/?link_id=a66ixg
https://api.pushshift.io/reddit/comment/search/?link_id=a674f7
https://api.pushshift.io/reddit/comment/search/?link_id=a675ch
https://api.pushshift.io/reddit/comment/search/?link_id=a67tq0
https://api.pushshift.io/reddit/comment/search/?link_id

https://api.pushshift.io/reddit/comment/search/?link_id=a79x1z
https://api.pushshift.io/reddit/comment/search/?link_id=a7a1h6
https://api.pushshift.io/reddit/comment/search/?link_id=a7a24v
https://api.pushshift.io/reddit/comment/search/?link_id=a7a3b4
https://api.pushshift.io/reddit/comment/search/?link_id=a7a3lz
https://api.pushshift.io/reddit/comment/search/?link_id=a7a5v4
https://api.pushshift.io/reddit/comment/search/?link_id=a7adie
https://api.pushshift.io/reddit/comment/search/?link_id=a7aeuf
https://api.pushshift.io/reddit/comment/search/?link_id=a7afzr
https://api.pushshift.io/reddit/comment/search/?link_id=a7aj15
https://api.pushshift.io/reddit/comment/search/?link_id=a7ank9
https://api.pushshift.io/reddit/comment/search/?link_id=a7aqkn
https://api.pushshift.io/reddit/comment/search/?link_id=a7asq4
https://api.pushshift.io/reddit/comment/search/?link_id=a7aus4
https://api.pushshift.io/reddit/comment/search/?link_id=a7av9q
https://api.pushshift.io/reddit/comment/search/?link_id

https://api.pushshift.io/reddit/comment/search/?link_id=a871qb
https://api.pushshift.io/reddit/comment/search/?link_id=a876po
https://api.pushshift.io/reddit/comment/search/?link_id=a87791
https://api.pushshift.io/reddit/comment/search/?link_id=a87lvd
https://api.pushshift.io/reddit/comment/search/?link_id=a87p8s
https://api.pushshift.io/reddit/comment/search/?link_id=a87ylb
https://api.pushshift.io/reddit/comment/search/?link_id=a880r2
https://api.pushshift.io/reddit/comment/search/?link_id=a884rf
https://api.pushshift.io/reddit/comment/search/?link_id=a88eo3
https://api.pushshift.io/reddit/comment/search/?link_id=a88n7k
https://api.pushshift.io/reddit/comment/search/?link_id=a88obg
https://api.pushshift.io/reddit/comment/search/?link_id=a88rmw
https://api.pushshift.io/reddit/comment/search/?link_id=a88rrj
https://api.pushshift.io/reddit/comment/search/?link_id=a89133
https://api.pushshift.io/reddit/comment/search/?link_id=a892h6
https://api.pushshift.io/reddit/comment/search/?link_id

https://api.pushshift.io/reddit/comment/search/?link_id=a9en1h
https://api.pushshift.io/reddit/comment/search/?link_id=a9enfu
https://api.pushshift.io/reddit/comment/search/?link_id=a9exkh
https://api.pushshift.io/reddit/comment/search/?link_id=a9f5qk
https://api.pushshift.io/reddit/comment/search/?link_id=a9f5y4
https://api.pushshift.io/reddit/comment/search/?link_id=a9fj1h
https://api.pushshift.io/reddit/comment/search/?link_id=a9fklk
https://api.pushshift.io/reddit/comment/search/?link_id=a9ft31
https://api.pushshift.io/reddit/comment/search/?link_id=a9g1jt
https://api.pushshift.io/reddit/comment/search/?link_id=a9gxvw
https://api.pushshift.io/reddit/comment/search/?link_id=a9hm3w
https://api.pushshift.io/reddit/comment/search/?link_id=a9hmfo
https://api.pushshift.io/reddit/comment/search/?link_id=a9hp6r
https://api.pushshift.io/reddit/comment/search/?link_id=a9jc8n
https://api.pushshift.io/reddit/comment/search/?link_id=a9jzoj
https://api.pushshift.io/reddit/comment/search/?link_id

https://api.pushshift.io/reddit/comment/search/?link_id=aad064
https://api.pushshift.io/reddit/comment/search/?link_id=aadl8c
https://api.pushshift.io/reddit/comment/search/?link_id=aador1
https://api.pushshift.io/reddit/comment/search/?link_id=aadv4s
https://api.pushshift.io/reddit/comment/search/?link_id=aah1ga
https://api.pushshift.io/reddit/comment/search/?link_id=aahbqs
https://api.pushshift.io/reddit/comment/search/?link_id=aahnb7
https://api.pushshift.io/reddit/comment/search/?link_id=aahwgd
https://api.pushshift.io/reddit/comment/search/?link_id=aai2y7
https://api.pushshift.io/reddit/comment/search/?link_id=aai32c
https://api.pushshift.io/reddit/comment/search/?link_id=aainjq
https://api.pushshift.io/reddit/comment/search/?link_id=aaioq5
https://api.pushshift.io/reddit/comment/search/?link_id=aaj67m
https://api.pushshift.io/reddit/comment/search/?link_id=aajaex
https://api.pushshift.io/reddit/comment/search/?link_id=aajbpf
https://api.pushshift.io/reddit/comment/search/?link_id

In [8]:
# Make list of comments into a single string
# submissions.comments = submissions.comments.apply(combine_comments)

In [9]:
submissions.head(1)

,subreddit,id,created_time,created_utc,title,selftext,num_comments,score,gilded,comments
0,gadgets,93jv9u,2018-08-01 00:01:18,1533081678,New INSANE Inventions That Exist in 2018 - 201...,,0,1,0.0,[]


In [10]:
# Construct output_df, which includes all the info we need from submissions
#output_df = submissions[['subreddit', 'id', 'created_time', 'created_utc', 'num_comments', 'score', 'gilded']]

# Combine the submission title, body, and comments into a single column called submission_text
#output_df['submission_text'] = (submissions['title'].map(str) + 
#                                submissions['selftext'].map(str) + 
#                                submissions['comments'].map(str))

# Rename id to submission_id (for clarity)
submissions.rename(columns={'id':'submission_id'}, inplace=True)

In [11]:
submissions.head(1)

,subreddit,submission_id,created_time,created_utc,title,selftext,num_comments,score,gilded,comments
0,gadgets,93jv9u,2018-08-01 00:01:18,1533081678,New INSANE Inventions That Exist in 2018 - 201...,,0,1,0.0,[]


In [12]:
# Pickle 
import pickle

# Pickle dataframe to use in other project file
with open('2018-08-01_to_2019-01-01', 'wb') as picklefile:
    pickle.dump(submissions, picklefile)